## PDLtools tutorial

### Database connectivity setup

In [122]:
%run 'database_connectivity_setup_demo.ipynb'
IPython.display.clear_output()
from IPython.display import display
import pandas as pd

### Toy dataset: autompg

In [78]:
%%showsql
select
    *
from
    autompg
limit 5;

,car_id,symboling,normalized_losses,make,fuel_type,aspiration,num_of_doors,body_style,drive_wheels,engine_location,wheel_base,length,width,height,curb_weight,engine_type,num_of_cylinders,engine_size,fuel_system,bore,stroke,compression_ratio,horsepower,peak_rpm,city_mpg,highway_mpg,price
0,31,0,102,subaru,gas,turbo,four,sedan,4wd,front,97.0,172.0,65.4,54.3,2510,ohcf,four,108,mpfi,3.62,2.64,7.7,111,4800,24,29,11259
1,63,-1,65,toyota,diesel,turbo,four,sedan,fwd,front,102.4,175.6,66.5,54.9,2480,ohc,four,110,idi,3.27,3.35,22.5,73,4500,30,33,10698
2,76,3,150,saab,gas,turbo,two,hatchback,fwd,front,99.1,186.6,66.5,56.1,2808,dohc,four,121,mpfi,3.54,3.07,9.0,160,5500,19,26,18150
3,108,1,128,nissan,diesel,std,two,sedan,fwd,front,94.5,165.3,63.8,54.5,2017,ohc,four,103,idi,2.99,3.47,21.9,55,4800,45,50,7099
4,25,-1,95,volvo,gas,turbo,four,sedan,rwd,front,109.1,188.8,68.9,55.5,3062,ohc,four,141,mpfi,3.78,3.15,9.5,114,5400,19,25,22625


In [81]:
%%showsql
select
    body_style,
    count(*)
from
    autompg
group by 1
order by 2 desc;

,body_style,count
0,sedan,79
1,hatchback,56
2,wagon,17
3,hardtop,5
4,convertible,2


In [91]:
%%showsql
select
    fuel_system,
    count(*)
from
    autompg
group by 1
order by 2 desc;

,fuel_system,count
0,mpfi,64
1,2bbl,63
2,idi,15
3,1bbl,11
4,spdi,5
5,mfi,1


In [147]:
sql = """
DROP TABLE IF EXISTS autompg_pivot;
SELECT sugarlib.sugar(
$$
CREATE TABLE autompg_pivot AS 
    SELECT 
        car_id,
        make,
        sugarlib.pivot01(body_style, autompg, null, null, sugarlib.all_but_one()),
        sugarlib.pivot01(fuel_system, autompg, null, null, sugarlib.freq_vals(3)),
        bore,
        stroke,
        compression_ratio,
        horsepower,
        num_of_doors,
        highway_mpg
    FROM 
        autompg
DISTRIBUTED by (car_id);
$$);
"""
psql.execute(sql, conn)
conn.commit()

In [148]:
%%showsql
select 
  *
from
  autompg_pivot
limit 5;

,car_id,make,body_style_convertible,body_style_hardtop,body_style_hatchback,body_style_wagon,fuel_system_mpfi,fuel_system_2bbl,fuel_system_idi,bore,stroke,compression_ratio,horsepower,num_of_doors,highway_mpg
0,32,porsche,0,0,1,0,1,0,0,3.94,3.11,9.5,143,two,27
1,64,mazda,0,0,0,0,0,1,0,3.39,3.39,8.6,84,four,32
2,25,volvo,0,0,0,0,1,0,0,3.78,3.15,9.5,114,four,25
3,149,honda,0,0,1,0,0,0,0,3.15,3.58,9.0,86,two,33
4,31,subaru,0,0,0,0,1,0,0,3.62,2.64,7.7,111,four,29


#### Anonymization

In [149]:
sql = """
DROP TABLE IF EXISTS anonymization_table;
SELECT pdltools.create_anonymization_table(
   'autompg_pivot', -- input table
   'anonymization_table', -- output table
   'make') -- column that contains the keys to anonymize
;"""
psql.execute(sql, conn)
conn.commit()

In [150]:
%%showsql
select * from anonymization_table;

,id,anon_id
0,volkswagen,e1b33361e5c98e10e4fd5182da1ea779
1,toyota,b87c422bc924a679cbf0865db6ec64ce
2,porsche,11252c989815273286f35cd7cc9692d1
3,audi,04a80a20346bfa3f29e61bb44dc9db0e
4,mercedes-benz,90703bd638a8fc188f23a91d54331307
5,peugot,71aa83deb1ebb9412ed27f3d11b18231
6,chevrolet,16eb2bbf59d8e6972430ac0eae951658
7,mitsubishi,891492fa329d93a482c8d125c00c4390
8,mazda,2a5fdb92487901e8bbd2a0b716394b6d
9,volvo,a55081a2c20499b4eabc0c398a0e554a


In [151]:
sql = """
DROP TABLE IF EXISTS anonymized_autompg;
SELECT pdltools.anonymize(
   'public', -- schema
   'autompg_pivot', -- input table
   'anonymized_autompg', -- output table (anonymized version of input table)
    array['make'], -- columns to anonymize
   'anonymization_table'); --anonymization table
"""
psql.execute(sql, conn)
conn.commit()

In [152]:
%%showsql
select 
  * 
from 
  anonymized_autompg
limit 5;

,car_id,make,body_style_convertible,body_style_hardtop,body_style_hatchback,body_style_wagon,fuel_system_mpfi,fuel_system_2bbl,fuel_system_idi,bore,stroke,compression_ratio,horsepower,num_of_doors,highway_mpg
0,129,e1b33361e5c98e10e4fd5182da1ea779,0,0,0,0,1,0,0,3.19,3.40,10.0,100,four,32
1,97,e1b33361e5c98e10e4fd5182da1ea779,0,0,0,0,1,0,0,3.19,3.40,9.0,85,two,34
2,22,a55081a2c20499b4eabc0c398a0e554a,0,0,0,0,1,0,0,3.78,3.15,9.5,114,four,28
3,54,71aa83deb1ebb9412ed27f3d11b18231,0,0,0,0,0,0,1,3.70,3.52,21.0,95,four,33
4,119,72a9f68525ba8d0ac8da7e8d87b3d769,0,0,0,0,0,1,0,2.97,3.23,9.4,68,four,38


In [153]:
%%showsql
select
    num_of_doors,
    count(*)
from
    anonymized_autompg
group by 1
order by 2 desc;

,num_of_doors,count
0,four,95
1,two,64


In [171]:
sql = """
DROP TABLE IF EXISTS stratified_autompg;
SELECT 
   pdltools.strat_partition(
      'anonymized_autompg', -- input table
      'stratified_autompg', -- output table
      'car_id, 
       make, 
       body_style_convertible, 
       body_style_hardtop, 
       body_style_hatchback,
       body_style_wagon, 
       fuel_system_mpfi, 
       fuel_system_2bbl, 
       fuel_system_idi, 
       bore, 
       stroke,
       compression_ratio,
       horsepower,
       highway_mpg', -- columns to reproduce in output table
      'num_of_doors', -- column to stratify by
      'split', -- name of the column that specifies the partition
      ARRAY[0.7, 0.3]
);
"""
psql.execute(sql, conn)
conn.commit()

In [172]:
%%showsql
select
    car_id,
    make,
    num_of_doors,
    split
from
    stratified_autompg
limit 10;

,car_id,make,num_of_doors,split
0,82,5afdc7e914bbb790a0a0f3421b4b5f90,two,1
1,5,891492fa329d93a482c8d125c00c4390,two,1
2,158,891492fa329d93a482c8d125c00c4390,two,0
3,126,b87c422bc924a679cbf0865db6ec64ce,two,0
4,49,b87c422bc924a679cbf0865db6ec64ce,two,0
5,132,e1b33361e5c98e10e4fd5182da1ea779,two,0
6,93,33d6488d7a0fb8e09acd80169fad6224,two,0
7,144,b87c422bc924a679cbf0865db6ec64ce,two,1
8,156,b87c422bc924a679cbf0865db6ec64ce,two,1
9,130,b87c422bc924a679cbf0865db6ec64ce,two,0


In [173]:
sql = """
DROP TABLE IF EXISTS autompg_train;
CREATE TABLE autompg_train AS (
    SELECT
      *
    FROM
      stratified_autompg
    WHERE
      split = 0
) distributed by (car_id);
"""
psql.execute(sql, conn)
conn.commit()

sql = """
DROP TABLE IF EXISTS autompg_test;
CREATE TABLE autompg_test AS (
    SELECT
      *
    FROM
      stratified_autompg
    WHERE
      split = 1
) distributed by (car_id);
"""
psql.execute(sql, conn)
conn.commit()

In [174]:
sql = """ 
DROP TABLE IF EXISTS autompg_mdl;
DROP TABLE IF EXISTS autompg_mdl_summary;
SELECT 
   madlib.linregr_train( 
      'autompg_train', -- input table
      'autompg_mdl', -- output table
      'highway_mpg', -- dependent variable (to predict)
      'ARRAY[1, 
            body_style_convertible, 
            body_style_hardtop, 
            body_style_hatchback, 
            body_style_wagon, 
            fuel_system_mpfi, 
            fuel_system_2bbl, 
            fuel_system_idi, 
            bore, 
            stroke,
            compression_ratio,
            horsepower ]' -- independent variables
);
"""
psql.execute(sql, conn)
conn.commit()

sql = """
DROP TABLE IF EXISTS autompg_linreg;
CREATE TABLE autompg_linreg
AS 
(
    SELECT 
       car_id,
       make,
       highway_mpg as y,
       madlib.linregr_predict( 
          ARRAY[1, 
                body_style_convertible, 
                body_style_hardtop, 
                body_style_hatchback, 
                body_style_wagon, 
                fuel_system_mpfi, 
                fuel_system_2bbl, 
                fuel_system_idi, 
                bore, 
                stroke,
                compression_ratio,
                horsepower],
            m.coef
        ) as y_hat
    FROM 
      autompg_test, autompg_mdl m
) distributed by (car_id)
"""
psql.execute(sql, conn)
conn.commit()

In [175]:
%%showsql
select 
  * 
from 
  autompg_linreg
limit 5;

,car_id,make,y,y_hat
0,129,e1b33361e5c98e10e4fd5182da1ea779,32,33.493809
1,111,72a9f68525ba8d0ac8da7e8d87b3d769,41,39.316339
2,31,33d6488d7a0fb8e09acd80169fad6224,29,24.118929
3,76,6b0b8f83681a7acdd8c952e24a7e6330,26,23.638188
4,34,6779c025a2c2a5644f18da328d175e81,38,38.087282


#### Prediction metrics

In [176]:
%%showsql
select 
  pdltools.mf_mae(y_hat,y) 
from
   autompg_linreg;

,mf_mae
0,2.480434


In [177]:
%%showsql
select 
  pdltools.mf_rmse(y_hat,y) 
from
   autompg_linreg;

,mf_rmse
0,3.454677


In [178]:
%%showsql
select 
  pdltools.mf_r2(y_hat,y) 
from
   autompg_linreg;

,mf_r2
0,0.64117


#### Stemming

In [16]:
%%showsql
select pdltools.stem_token('run')

,stem_token
0,run


In [17]:
%%showsql
select pdltools.stem_token('running')

,stem_token
0,run


In [37]:
%%showsql
select pdltools.stem_token_arr(ARRAY['I','walked','and','played','with','friendly','dogs'])

,stem_token_arr
0,"[I, walk, and, plai, with, friendli, dog]"


#### Edit distance

In [54]:
%%showsql
select pdltools.edit_distance(
    'AGGCTATCACCTGACCTCCAGGCCGATGCCC', -- string1 
    'TAGCTATCACGACCGCGGTCGATTTGCCCGA', -- string2
    1, -- insert cost 
    1, -- delete cost 
    2, -- substitution cost
    2 -- transpose cost
);

,edit_distance
0,14


In [55]:
%%showsql
select pdltools.optimal_alignment_distance(
    'AGGCTATCACCTGACCTCCAGGCCGATGCCC', -- string1 
    'TAGCTATCACGACCGCGGTCGATTTGCCCGA' -- string2
);

,optimal_alignment_distance
0,12


#### Complex numbers

Constructor

In [179]:
%%showsql
select 
   pdltools.complex(5,3);

,complex
0,5 + 3i


Retrieving a complex number's real part.

In [182]:
%%showsql
select
   pdltools.re(pdltools.complex(5,3)), pdltools.im(pdltools.complex(5,3));

,re,im
0,5,3


Angle from the positive real line in radians.

In [184]:
%%showsql
select
   pdltools.radians(pdltools.complex(1,sqrt(3)));

,radians
0,1.047198
